<a href="https://colab.research.google.com/github/adamstiefel/AI-Business-Agents/blob/main/Business_Model_Canvas_(BMC)_Generator_%26_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💡 Business Model Canvas (BMC) Generator & Advisor 🚀

## Goal
To take a user's business concept as input, use a Large Language Model (LLM) to generate a structured Business Model Canvas (listing key hypotheses/ideas for each of the 9 blocks), and provide strategic recommendations.

## How this "Makes Money" (Delivers Business Value)

* 🚀 **Accelerate Ideation & Validation:** Quickly transforms a raw business idea into a structured model, helping founders, innovators, and product managers clarify their thinking and identify core components that need validation.
* 🤔 **Framework for Strategic Thinking:** Provides a holistic view of the potential business, highlighting interdependencies, areas needing more thought, and potential gaps.
* 🗣️ **Improve Pitching & Communication:** A fleshed-out BMC is a powerful and concise tool for communicating a business idea to potential investors, partners, team members, or stakeholders.
* 🎯 **Identify Key Assumptions & Risks:** The process of generating the BMC helps uncover critical underlying assumptions that need to be tested and potential business model risks that need mitigation.
* 📝 **Foundation for Business Planning:** Serves as a strong and quick starting point for developing a more detailed business plan, saving significant initial drafting time.
* 💰 **Reduce Early-Stage "Trial and Error" Costs:** By thinking through the model components upfront, businesses can potentially avoid costly missteps and pivot more effectively.
* ✨ **Spark Innovation & Exploration:** Can be used to quickly sketch out and compare different business model options for a single concept or explore new ventures.
* 🤝 **Facilitate Team Alignment:** Provides a common framework for team members to discuss and align on the core aspects of a new business or product.

---
**Important Note:** This AI agent generates a *first-pass* Business Model Canvas and strategic suggestions based on the provided concept. The output should be seen as a **hypothesis-generation tool and a thought-starter**. It requires critical review, further research, validation, and refinement by experienced humans.
---

In [4]:
#@title 2. Setup and Installations
# --- Install necessary libraries ---
!pip install openai -q

import openai
from IPython.display import display, Markdown # For formatted output
import json # If we need to structure output more formally later

print("Libraries installed/available and imported.")

Libraries installed/available and imported.


In [5]:
#@title 3. OpenAI API Key Configuration
# --- Configure your OpenAI API Key ---
# Recommended: Use Colab Secrets (Key icon on the left sidebar)
# Add a secret named 'OPENAI_API_KEY' with your actual OpenAI API key.
from google.colab import userdata

OPENAI_API_KEY = None # Initialize
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        openai.api_key = OPENAI_API_KEY
        print("✅ OpenAI API Key loaded successfully from Colab Secrets.")
    else:
        print("⚠️ OpenAI API Key not found in Colab Secrets. Please add it.")
        # Fallback for manual input if needed, though not recommended for shared notebooks
        # OPENAI_API_KEY_MANUAL = "" # PASTE YOUR KEY HERE
        # if OPENAI_API_KEY_MANUAL:
        #    openai.api_key = OPENAI_API_KEY_MANUAL
        #    OPENAI_API_KEY = OPENAI_API_KEY_MANUAL # Ensure global var is set
        #    print("✅ OpenAI API Key set directly (less secure).")
        # else:
        #    print("🛑 API Key not provided manually either.")

except Exception as e:
    print(f"⚠️ Error accessing Colab Secrets: {e}")
    print("Please ensure you have an OpenAI API key. You might need to set it manually if Colab Secrets are unavailable in your environment.")

if not openai.api_key: # Check if it was actually set
    print("🛑 OpenAI API Key is NOT SET. LLM functionality will not work.")

✅ OpenAI API Key loaded successfully from Colab Secrets.


## Quick Guide: The 9 Building Blocks of the Business Model Canvas

The Business Model Canvas (BMC) helps you describe, design, challenge, and pivot your business model. This agent will attempt to generate ideas for each of these 9 blocks based on your concept:

1.  **Customer Segments:** Who are your most important customers? What are their archetypes?
2.  **Value Propositions:** What value do you deliver to the customer? Which customer problem are you solving, or which need are you satisfying? What is unique about your offering?
3.  **Channels:** How do you reach your Customer Segments to deliver your Value Proposition? (e.g., sales force, web sales, own stores, partner stores, wholesale).
4.  **Customer Relationships:** What type of relationship does each Customer Segment expect you to establish and maintain with them? (e.g., personal assistance, self-service, automated, community).
5.  **Revenue Streams:** How do you generate cash from each Customer Segment? (e.g., asset sale, usage fee, subscription, licensing, advertising).
6.  **Key Activities:** What are the most important things your company must *do* to make its business model work? (e.g., production, problem-solving, platform/network).
7.  **Key Resources:** What are the most important *assets* required to make your business model work? (e.g., physical, intellectual, human, financial).
8.  **Key Partners:** Who are the key partners and suppliers needed to make the business model work? (e.g., strategic alliances, joint ventures, buyer-supplier relationships).
9.  **Cost Structure:** What are the most important costs incurred while operating under your business model? (Are you cost-driven or value-driven?).

In [6]:
#@title 5. Input: Describe Your Business Concept
#@markdown Please provide a clear and concise description of your business idea or concept.
#@markdown The more detail you provide about the problem you're solving, your proposed solution, target audience, and unique aspects, the better the generated BMC and recommendations will be.

business_concept_input = """
Example: A subscription box service for artisanal coffee lovers. Each month, subscribers receive a curated selection of whole bean coffee from unique micro-roasters around the world, along with tasting notes and brewing tips. The service aims to help coffee enthusiasts discover new flavors and support small-batch roasters. We plan to build a strong online community.
""" #@param {type:"string"}

if not business_concept_input.strip() or \
   ("Example: A subscription box service for artisanal coffee lovers." in business_concept_input and len(business_concept_input) < 250): # Basic check for default
    print("⚠️ Please replace the example business concept with your own idea above.")
else:
    print("✅ Business concept received.")

✅ Business concept received.


In [7]:
#@title 6. Generate Business Model Canvas & Recommendations

#@markdown Select the LLM model to use (GPT-4 class models are STRONGLY recommended for this complex task):
llm_model_bmc = "gpt-4-turbo-preview" #@param ["gpt-4-turbo-preview", "gpt-4", "gpt-3.5-turbo"]
#@markdown *Ensure your API key supports the selected model.*

def generate_bmc_and_recommendations(concept, model_name="gpt-4-turbo-preview"):
    if not openai.api_key:
        return "Error: OpenAI API Key not set. Please configure it in Step 3."
    if not concept or concept.strip() == "" or \
       ("Example: A subscription box service for artisanal coffee lovers." in concept and len(concept) < 250) :
        return "Error: Please provide a valid and detailed business concept in Step 5."

    prompt = f"""
    You are an expert Business Strategist and Innovation Consultant.
    Your task is to analyze the following business concept and generate a comprehensive Business Model Canvas (BMC) for it, followed by key strategic recommendations.

    Business Concept:
    ---
    {concept}
    ---

    Please structure your output clearly using Markdown. For each of the 9 blocks of the Business Model Canvas, provide 3-5 key hypotheses or ideas based *only* on the provided concept. If information for a specific block is not directly derivable from the concept, make reasonable inferences or state what needs to be defined.

    Output Format:

    ## Business Model Canvas for: [Infer a name for the business concept if possible]

    ### 1. Customer Segments
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 2. Value Propositions
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 3. Channels
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 4. Customer Relationships
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 5. Revenue Streams
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 6. Key Activities
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 7. Key Resources
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 8. Key Partners
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ### 9. Cost Structure
    * [Hypothesis/Idea 1]
    * [Hypothesis/Idea 2]
    * ...

    ## Strategic Recommendations
    Based on the generated BMC and the initial concept, provide 3-5 key strategic recommendations. These could include:
    * Critical assumptions to test first.
    * Potential competitive advantages to build upon.
    * Key risks to mitigate.
    * Immediate next steps for developing the business model.
    * [Recommendation 1 with brief justification]
    * [Recommendation 2 with brief justification]
    * ...

    Be insightful, practical, and ensure your analysis is grounded in the provided business concept.
    """

    print(f"\n🤖 Generating Business Model Canvas and Recommendations using {model_name}...")
    print("This may take a moment, especially with advanced models and detailed concepts...")

    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are an expert Business Strategist helping to flesh out a new business concept into a Business Model Canvas and provide recommendations."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6, # Slightly higher temperature for creative brainstorming within the BMC blocks
            max_tokens=3000  # Allow for a detailed BMC and recommendations
        )
        bmc_report = response.choices[0].message.content
        print("✅ Business Model Canvas and recommendations generated.")
        return bmc_report
    except openai.APIError as e:
        return f"OpenAI API Error: {e}. Check your API key, model access, or usage limits."
    except Exception as e:
        return f"An unexpected error occurred during LLM call: {e}"

# --- Main execution for this cell ---
is_default_concept = "Example: A subscription box service for artisanal coffee lovers." in business_concept_input and len(business_concept_input) < (len("Example: A subscription box service for artisanal coffee lovers.") + 150)

if 'business_concept_input' in globals() and business_concept_input.strip() and not is_default_concept:
    if openai.api_key: # Check if API key is actually set
        generated_bmc_report = generate_bmc_and_recommendations(
            business_concept_input,
            model_name=llm_model_bmc
        )
        if generated_bmc_report.startswith("Error:"):
            print(f"\n{generated_bmc_report}")
            display(Markdown(f"**{generated_bmc_report}**"))
        else:
            print("\n--- Generated Business Model Canvas & Strategic Recommendations ---")
            display(Markdown(generated_bmc_report))
    else:
        error_message_bmc = "🛑 OpenAI API Key not configured. Please set it up in Step 3."
        print(f"\n{error_message_bmc}")
        display(Markdown(f"**{error_message_bmc}**"))
else:
    error_message_bmc = "⚠️ Please provide your unique business concept in Step 5 before generating the BMC. The current input might be empty or too similar to the example."
    print(f"\n{error_message_bmc}")
    display(Markdown(f"**{error_message_bmc}**"))


🤖 Generating Business Model Canvas and Recommendations using gpt-4-turbo-preview...
This may take a moment, especially with advanced models and detailed concepts...
✅ Business Model Canvas and recommendations generated.

--- Generated Business Model Canvas & Strategic Recommendations ---


## Business Model Canvas for: WorldCrafter Coffees

### 1. Customer Segments
* **Coffee enthusiasts** who value unique, high-quality brews and are willing to pay a premium.
* **Supporters of small businesses** looking to support independent and micro-roasters.
* **Gift buyers** seeking unique presents for coffee lovers in their lives.
* **Subscription box collectors** who enjoy the surprise element and discovering new products.

### 2. Value Propositions
* **Exclusive access** to unique, artisanal coffee from around the world not easily found in stores.
* **Educational content** including tasting notes and brewing tips to enhance the coffee experience.
* **Support for small-batch roasters** by providing them with a wider audience.
* **Community engagement** through an online platform for sharing experiences and recommendations.

### 3. Channels
* **E-commerce website** for subscriptions, customer service, and community engagement.
* **Social media platforms** (Instagram, Facebook) to market the service, share customer experiences, and build the online community.
* **Email newsletters** for personalized recommendations, brewing tips, and community highlights.

### 4. Customer Relationships
* **Personalized customer service** with coffee recommendations and brewing support.
* **Community-driven content**, encouraging subscribers to share their tasting notes and brewing experiences.
* **Exclusive events**, such as virtual tastings with roasters or brewing workshops.

### 5. Revenue Streams
* **Monthly subscription fees**, with options for different tiers based on quantity or exclusivity of coffee selections.
* **Gift subscriptions** for single months or a full year.
* **Affiliate sales** of brewing equipment or branded merchandise.

### 6. Key Activities
* **Curating coffee selections** from different micro-roasters each month.
* **Creating educational and engaging content** for the subscribers.
* **Building and maintaining the online community** and e-commerce platform.
* **Managing relationships** with micro-roasters and logistics partners.

### 7. Key Resources
* **Relationships with micro-roasters** worldwide to ensure a diverse and high-quality selection.
* **A robust e-commerce platform** that can handle subscriptions, community engagement, and content delivery.
* **Content creation team** for producing educational and promotional materials.
* **Logistics network** to manage global shipping efficiently.

### 8. Key Partners
* **Micro-roasters** for supplying artisanal coffee.
* **Logistics companies** for handling shipping and fulfillment.
* **Coffee equipment suppliers** for affiliate marketing or bundled sales.
* **Coffee experts** or influencers for content creation and community engagement.

### 9. Cost Structure
* **Procurement of coffee** from micro-roasters, including shipping costs to the central facility.
* **Shipping and handling costs** for sending out subscription boxes.
* **Platform development and maintenance** including e-commerce and community features.
* **Content creation and marketing expenses** to engage and grow the subscriber base.
* **Customer service operations**, including staff and technology to support subscribers.

## Strategic Recommendations

1. **Validate the interest and purchasing power** of the target customer segments through market research and early sign-ups. This will ensure there's a viable market willing to pay for the proposed value propositions.
   
2. **Build relationships with micro-roasters** by highlighting the mutual benefits. For roasters, emphasize the exposure and sales opportunities. For WorldCrafter Coffees, ensure a steady supply of unique, quality products.

3. **Leverage social proof and community engagement** as a competitive advantage. Encourage user-generated content, such as unboxing videos or brewing demonstrations, to foster a sense of community and trust in the brand.

4. **Develop a robust logistics strategy** to manage the complexities of shipping fresh coffee worldwide. This includes negotiating favorable rates with logistics partners and ensuring compliance with international shipping regulations for food products.

5. **Test different subscription tiers and offerings** early on to understand customer preferences. This could include variations in quantity, exclusivity of selections, or even options for ground versus whole bean coffee for different brewing preferences.

By focusing on these strategic areas, WorldCrafter Coffees can build a strong foundation for a successful subscription service that delights coffee enthusiasts and supports small-batch roasters globally.